In [36]:
print("hello")

hello


In [33]:
import numpy
import os
import pandas
import glob
import time

import argparse
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions

In [34]:
class MLP(chainer.Chain):

    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)  # n_units -> n_out

    def forward(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)


In [3]:
parser = argparse.ArgumentParser(description='Chainer example: MNIST')
parser.add_argument('--batchsize', '-b', type=int, default=100,
                        help='Number of images in each mini-batch')
parser.add_argument('--epoch', '-e', type=int, default=20,
                        help='Number of sweeps over the dataset to train')
parser.add_argument('--frequency', '-f', type=int, default=-1,
                        help='Frequency of taking a snapshot')
parser.add_argument('--gpu', '-g', type=int, default=-1,
                        help='GPU ID (negative value indicates CPU)')
parser.add_argument('--out', '-o', default='result',
                        help='Directory to output the result')
parser.add_argument('--resume', '-r', default='',
                        help='Resume the training from snapshot')
parser.add_argument('--unit', '-u', type=int, default=1000,
                        help='Number of units')
parser.add_argument('--noplot', dest='plot', action='store_false',
                        help='Disable PlotReport extension')
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--batchsize BATCHSIZE] [--epoch EPOCH]
                             [--frequency FREQUENCY] [--gpu GPU] [--out OUT]
                             [--resume RESUME] [--unit UNIT] [--noplot]
ipykernel_launcher.py: error: argument --frequency/-f: invalid int value: 'C:\\Users\\Owner\\AppData\\Roaming\\jupyter\\runtime\\kernel-dd0ea1ca-58b0-41fa-9c84-b6419d03e174.json'


SystemExit: 2

C:\Users\Owner\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [35]:
model = L.Classifier(MLP(1000, 1),  # 回帰用に修正: 1つの出力
                         lossfun=F.mean_squared_error,  # 回帰用に修正; MSE
                         accfun=F.r2_score  # 回帰用に修正; 相関係数R2
                         )

In [36]:
optimizer = chainer.optimizers.SGD()
optimizer.setup(model)

In [37]:
train, test = chainer.datasets.get_mnist()

In [11]:
print(train)

In [38]:
import numpy
train = chainer.datasets.tuple_dataset.TupleDataset(
        train.__dict__["_datasets"][0],
        train.__dict__["_datasets"][1].astype(numpy.float32).reshape(-1, 1))

test = chainer.datasets.tuple_dataset.TupleDataset(
        test.__dict__["_datasets"][0],
        test.__dict__["_datasets"][1].astype(numpy.float32).reshape(-1, 1))

In [39]:
train_iter = chainer.iterators.SerialIterator(train, 100)

In [40]:
test_iter = chainer.iterators.SerialIterator(test, 20,
                                                 repeat=False, shuffle=False)

In [15]:
updater = training.updaters.StandardUpdater(
        train_iter, optimizer, device=-1)

In [42]:
# いつも使っているやーつ
updater = training.updaters.StandardUpdater(train_iter, optimizer, device=-1)
trainer = training.Trainer(updater, (20, 'epoch'), out="Result2018_oono/%s" % time.strftime("%Y%m%d%H%M%S"))

In [45]:
# Evaluate the model with the test dataset for each epoch
trainer.extend(extensions.Evaluator(test_iter, model))

# Dump a computational graph from 'loss' variable at the first iteration
# The "main" refers to the target link of the "main" optimizer.
trainer.extend(extensions.dump_graph('main/loss'))

# Take a snapshot for each specified epoch
# frequency = args.epoch if args.frequency == -1 else max(1, args.frequency)
# trainer.extend(extensions.snapshot(), trigger=(frequency, 'epoch'))

# Write a log of evaluation statistics for each epoch
trainer.extend(extensions.LogReport())

# Save two plot images to the result dir
trainer.extend(
    extensions.PlotReport(['main/loss', 'validation/main/loss'],
                          'epoch', file_name='loss.png'))
trainer.extend(
    extensions.PlotReport(
        ['main/accuracy', 'validation/main/accuracy'],
        'epoch', file_name='accuracy.png'))

# Print selected entries of the log to stdout
# Here "main" refers to the target link of the "main" optimizer again, and
# "validation" refers to the default name of the Evaluator extension.
# Entries other than 'epoch' are reported by the Classifier link, called by
# either the updater or the evaluator.
trainer.extend(extensions.PrintReport(
    ['epoch', 'main/loss', 'validation/main/loss',
     'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

# Print a progress bar to stdout
trainer.extend(extensions.ProgressBar())

# if args.resume:
#     # Resume from a snapshot
#     chainer.serializers.load_npz(args.resume, trainer)


In [19]:
trainer.extend(extensions.Evaluator(test_iter, model, device=-1))

In [20]:
trainer.extend(extensions.dump_graph('main/loss'))

In [22]:
trainer.extend(extensions.PlotReport(['main/loss', 'validation/main/loss'],'epoch', file_name='loss.png'))

In [23]:
trainer.extend(extensions.PlotReport(['main/accuracy', 'validation/main/accuracy'],'epoch', file_name='accuracy.png'))

In [24]:
trainer.extend(extensions.PrintReport(
        ['epoch', 'main/loss', 'validation/main/loss',
         'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

In [25]:
trainer.extend(extensions.ProgressBar())

In [46]:
trainer.run()


epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
     total [..................................................]  0.83%
this epoch [########..........................................] 16.67%
       100 iter, 0 epoch / 20 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [..................................................]  0.83%
this epoch [########..........................................] 16.67%
       100 iter, 0 epoch / 20 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [..................................................]  1.67%
this epoch [################..................................] 33.33%
       200 iter, 0 epoch / 20 epochs
    25.361 iters/sec. Estimated time to finish: 0:07:45.280874.
     total [..................................................]  1.67%
this epoch [################..................................] 33.33%
       200 iter, 0 epoch / 20 epochs
    25

     total [#######...........................................] 14.17%
this epoch [#########################################.........] 83.33%
      1700 iter, 2 epoch / 20 epochs
    26.021 iters/sec. Estimated time to finish: 0:06:35.838687.
     total [#######...........................................] 14.17%
this epoch [#########################################.........] 83.33%
      1700 iter, 2 epoch / 20 epochs
    26.021 iters/sec. Estimated time to finish: 0:06:35.838676.
     total [#######...........................................] 15.00%
this epoch [..................................................]  0.00%
      1800 iter, 3 epoch / 20 epochs
    24.158 iters/sec. Estimated time to finish: 0:07:02.226235.
3           0.867577    0.796482              0.89441        0.898221                  74.3765       
     total [#######...........................................] 15.00%
this epoch [..................................................]  0.00%
      1800 iter, 3 epoch / 

     total [#############.....................................] 27.50%
this epoch [#########################.........................] 50.00%
      3300 iter, 5 epoch / 20 epochs
    24.734 iters/sec. Estimated time to finish: 0:05:51.748060.
     total [#############.....................................] 27.50%
this epoch [#########################.........................] 50.00%
      3300 iter, 5 epoch / 20 epochs
    24.733 iters/sec. Estimated time to finish: 0:05:51.750751.
     total [##############....................................] 28.33%
this epoch [#################################.................] 66.67%
      3400 iter, 5 epoch / 20 epochs
    24.812 iters/sec. Estimated time to finish: 0:05:46.602717.
     total [##############....................................] 28.33%
this epoch [#################################.................] 66.67%
      3400 iter, 5 epoch / 20 epochs
    24.812 iters/sec. Estimated time to finish: 0:05:46.600115.
     total [##############..

8           0.428152    0.562905              0.947887       0.928153                  205.301       
     total [####################..............................] 40.83%
this epoch [########..........................................] 16.67%
      4900 iter, 8 epoch / 20 epochs
    23.477 iters/sec. Estimated time to finish: 0:05:02.417773.
     total [####################..............................] 40.83%
this epoch [########..........................................] 16.67%
      4900 iter, 8 epoch / 20 epochs
    23.478 iters/sec. Estimated time to finish: 0:05:02.416295.
     total [####################..............................] 41.67%
this epoch [################..................................] 33.33%
      5000 iter, 8 epoch / 20 epochs
    23.636 iters/sec. Estimated time to finish: 0:04:56.160528.
     total [####################..............................] 41.67%
this epoch [################..................................] 33.33%
      5000 iter, 8 epoch / 

     total [###########################.......................] 54.17%
this epoch [#########################################.........] 83.33%
      6500 iter, 10 epoch / 20 epochs
    23.956 iters/sec. Estimated time to finish: 0:03:49.583132.
     total [###########################.......................] 54.17%
this epoch [#########################################.........] 83.33%
      6500 iter, 10 epoch / 20 epochs
    23.957 iters/sec. Estimated time to finish: 0:03:49.579703.
     total [###########################.......................] 55.00%
this epoch [..................................................]  0.00%
      6600 iter, 11 epoch / 20 epochs
    23.597 iters/sec. Estimated time to finish: 0:03:48.845395.
11          0.318686    0.513767              0.961291       0.934489                  279.463       
     total [###########################.......................] 55.00%
this epoch [..................................................]  0.00%
      6600 iter, 11 epoc

     total [#################################.................] 67.50%
this epoch [#########################.........................] 50.00%
      8100 iter, 13 epoch / 20 epochs
    24.107 iters/sec. Estimated time to finish: 0:02:41.780380.
     total [#################################.................] 67.50%
this epoch [#########################.........................] 50.00%
      8100 iter, 13 epoch / 20 epochs
    24.107 iters/sec. Estimated time to finish: 0:02:41.778434.
     total [##################################................] 68.33%
this epoch [#################################.................] 66.67%
      8200 iter, 13 epoch / 20 epochs
    24.129 iters/sec. Estimated time to finish: 0:02:37.483939.
     total [##################################................] 68.33%
this epoch [#################################.................] 66.67%
      8200 iter, 13 epoch / 20 epochs
     24.13 iters/sec. Estimated time to finish: 0:02:37.482068.
     total [############

16          0.212624    0.491624              0.974002       0.937338                  396.441       
     total [########################################..........] 80.83%
this epoch [########..........................................] 16.67%
      9700 iter, 16 epoch / 20 epochs
    24.228 iters/sec. Estimated time to finish: 0:01:34.929985.
     total [########################################..........] 80.83%
this epoch [########..........................................] 16.67%
      9700 iter, 16 epoch / 20 epochs
    24.229 iters/sec. Estimated time to finish: 0:01:34.928789.
     total [########################################..........] 81.67%
this epoch [################..................................] 33.33%
      9800 iter, 16 epoch / 20 epochs
    24.323 iters/sec. Estimated time to finish: 0:01:30.451209.
     total [########################################..........] 81.67%
this epoch [################..................................] 33.33%
      9800 iter, 16 epoc

     total [###############################################...] 94.17%
this epoch [#########################################.........] 83.33%
     11300 iter, 18 epoch / 20 epochs
    24.678 iters/sec. Estimated time to finish: 0:00:28.365555.
     total [###############################################...] 94.17%
this epoch [#########################################.........] 83.33%
     11300 iter, 18 epoch / 20 epochs
    24.678 iters/sec. Estimated time to finish: 0:00:28.365346.
     total [###############################################...] 95.00%
this epoch [..................................................]  0.00%
     11400 iter, 19 epoch / 20 epochs
    24.489 iters/sec. Estimated time to finish: 0:00:24.500598.
19          0.176176    0.460428              0.978555       0.941164                  464.724       
     total [###############################################...] 95.00%
this epoch [..................................................]  0.00%
     11400 iter, 19 epoc